In [19]:
from ipycanvas import Canvas
from ipywidgets import Output
import numpy as np
from ipywidgets import interact
import cv2 as cv
import va
import ipywidgets as widgets
from IPython.display import display

In [20]:
w,h,bs = 0,0,0
mtx = np.zeros((w,h),dtype=np.uint8)

@interact(brush_size=(2,32),width=(100,1024),height=(100,1024))
def set_param(brush_size=32,width=256,height=256):
    global w,h,bs,mtx
    bs = brush_size
    w = width
    h = height
    mtx = np.zeros((w,h),dtype=np.uint8)

interactive(children=(IntSlider(value=32, description='brush_size', max=32, min=2), IntSlider(value=256, descr…

In [26]:
canvas = Canvas(width=w, height=h)
canvas.put_image_data(mtx, 0, 0)
mtx = np.zeros((w,h),dtype=np.uint16)
out = Output()
reg = False

@out.capture()
def handle_mouse_move(x, y):
    global reg
    if reg :
        y, x = int(y), int(x)
        bs2 = bs//2
        mtx[y-bs2:y+bs2+1,x-bs2:x+bs2+1] = 255
        canvas.put_image_data(mtx, 0, 0)

@out.capture()
def handle_mouse_down(x, y):
    global reg
    reg = True

@out.capture()
def handle_mouse_up(x, y):
    global reg
    reg = False
    
def btn_run_nn(arg):
    global mtx
    net = cv.dnn.readNet('minst/model.onnx')
    mtx_resized = cv.resize(mtx, (28, 28), interpolation=cv.INTER_LINEAR).astype(np.uint8)
    v = cv.dnn.blobFromImage(mtx_resized)
    net.setInput(v)
    nn_out = net.forward()
    classe = np.argmax(nn_out)
    print(f"classe predetta = {classe}")
    mtx = np.zeros((w,h),dtype=np.uint8)
    canvas.put_image_data(mtx, 0, 0)

canvas.on_mouse_move(handle_mouse_move)
canvas.on_mouse_down(handle_mouse_down)
canvas.on_mouse_up(handle_mouse_up)

display(out)
canvas

Output()

Canvas(height=412, width=412)

In [25]:
button_run_nn = widgets.Button(description = 'Run Neural Network')   
button_run_nn.on_click(btn_run_nn)
display(button_run_nn)

Button(description='Run Neural Network', style=ButtonStyle())